In [1]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

raw_data_dir = 'datasets\sales_train_evaluation.csv'
processed_data_dir = 'datasets\\'
ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'

STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 0                  
END_TRAIN   = 1941
P_HORIZON   = 28

mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

remove_features = [TARGET,'id','state_id','store_id',
                    'item_id', 'dept_id', 'cat_id','date','wm_yr_wk','d', \
                    'release', \
                    'price_norm', 'price_nunique', 'item_nunique', \
                    'price_momentum', \
                    'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', \
                    'enc_cat_id_std','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', \
                    'enc_dept_id_std', 'enc_item_id_std', \
                    'rolling_std_7', 'rolling_std_14', \
                    'rolling_std_30', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', \
                    'rolling_mean_tmp_1_30', 'rolling_mean_tmp_7_7', \
                    'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', \
                    'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', \
                    'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', \
                    'rolling_mean_7', 'rolling_mean_14', 'rolling_mean_30', 'rolling_mean_60', 'rolling_std_60', 'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_60', \
                    'rolling_mean_tmp_14_30']

In [2]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    

    df = df[df['d']>=START_TRAIN]
    
    df = df[df['store_id']==store]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3

    state = "snap_" + store.split('_')[0]
    states = ['snap_CA','snap_TX','snap_WI']
    deleted_states = []
    for i in states:
        if i != state:
            deleted_states.append(i)
            
    features = [col for col in list(df) if (col not in remove_features and col not in deleted_states)]
    
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test

# split(trainning and validation)
def split(dataframe_for_splitting):
    train = dataframe_for_splitting[dataframe_for_splitting["d"] <= END_TRAIN - P_HORIZON]
    validation = dataframe_for_splitting[(dataframe_for_splitting["d"] <= END_TRAIN) & (dataframe_for_splitting["d"] > END_TRAIN - P_HORIZON)]
    return train, validation

In [3]:
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

n_estimators = 150  # Number of boosting stages (iterations)
learning_rate = 0.01  # Shrinkage rate

for i in range(len(STORES_IDS)):
    df, features = get_data_by_store(STORES_IDS[i])
    features.append('sales')
    features.append('d')
    columns_to_remove = ['id']
    features = list(df.columns)
    for col in columns_to_remove:
        features.remove(col)
    
    df = df[features]
    df.dropna(inplace=True)
    train_df, validation_df = split(df)
    features.remove('d')
    train_df = train_df[features]
    validation_df = validation_df[features]

    X_train = train_df.drop(columns=['sales'])
    y_train = train_df['sales']

    X_val = validation_df.drop(columns=['sales'])
    y_val = validation_df['sales']

    gb_model = GradientBoostingRegressor(loss='squared_error', 
                                         n_estimators=n_estimators, 
                                         learning_rate=learning_rate,
                                         random_state=66, 
                                         verbose=1)

    gb_model.fit(X_train, y_train)

    # Compute loss values for each boosting stage on the training set
    train_loss = [mean_squared_error(y_train, y_pred) for y_pred in gb_model.staged_predict(X_train)]
    
    # Compute loss values for each boosting stage on the validation set
    val_loss = [mean_squared_error(y_val, y_pred) for y_pred in gb_model.staged_predict(X_val)]

    # Save the trained model to a file
    model_filename = f'model_store_{STORES_IDS[i]}_GB_0218.joblib'
    joblib.dump(gb_model, model_filename)
    print(f"Model for store {STORES_IDS[i]} saved as {model_filename}")

    # Print loss values for each boosting stage
    for stage, (train_loss_value, val_loss_value) in enumerate(zip(train_loss, val_loss), start=1):
        print(f"Iteration {stage}: Training Loss = {train_loss_value}, Validation Loss = {val_loss_value}")

    y_val_pred = gb_model.predict(X_val)

    mse = mean_squared_error(y_val, y_val_pred)
    print("Mean Squared Error on Validation Set:", mse)


      Iter       Train Loss   Remaining Time 
         1          19.6576           25.78m
         2          19.4772           25.47m
         3          19.3003           25.17m
         4          19.1269           25.12m
         5          18.9524           25.19m
         6          18.7853           25.06m
         7          18.6173           24.84m
         8          18.4565           24.67m
         9          18.2986           24.44m
        10          18.1389           24.26m
        20          16.7097           30.32m
        30          15.5040           31.31m
        40          14.4904           30.27m
        50          13.6251           28.10m
        60          12.8862           25.88m
        70          12.2547           23.38m
        80          11.7187           20.70m
        90          11.2605           17.90m
       100          10.8685           15.02m
Model for store CA_1 saved as model_store_CA_1_GB_0218.joblib
Iteration 1: Training Loss = 19.65761

In [ ]:
print(df.columns)

In [ ]:
import joblib
from sklearn.impute import SimpleImputer
import pandas as pd

predicted_df = pd.DataFrame(columns=['id', 'prediction'])

for i in range(len(STORES_IDS)):
    # Load the saved model
    loaded_model = joblib.load(f'model_store_{STORES_IDS[i]}_GB.joblib')

    df, _ = get_data_by_store(STORES_IDS[i])
    df = pd.get_dummies(df, columns=['event_name_1'])
    columns_to_remove = ['id']
    features = list(df.columns)
    for col in columns_to_remove:
        features.remove(col)
    test = df[df["d"] > END_TRAIN]
    id = test["id"]
    features.remove('d')
    X_test = test[features]
    X_test = X_test.drop(columns=['sales'])
    
    imputer = SimpleImputer(strategy='median')  # You can choose 'mean', 'median', 'most_frequent', or a constant value
    X_test = imputer.fit_transform(X_test)
    # Use the loaded model for prediction
    prediction = loaded_model.predict(X_test)

    print(f"--Predition of {STORES_IDS[i]}--")
    print(id)
    print(prediction)

    temp_df = pd.DataFrame({'id': id, 'prediction': prediction})
    temp_df.to_csv(f"results_GB_{STORES_IDS[i]}_0218.csv",index=False)
    print(f"Predicted Time Ahead: {len(temp_df)}")
    # Concatenate the temporary DataFrame to the main DataFrame along the row direction
    predicted_df = pd.concat([predicted_df, temp_df])
    predicted_df.reset_index(drop=True, inplace=True)

    


In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

raw_data_dir = 'datasets\sales_train_evaluation.csv'
processed_data_dir = 'datasets\\'
ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'

STORES_IDS = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 0                  
END_TRAIN   = 1941
P_HORIZON   = 28

mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

remove_features = [TARGET,'id','state_id','store_id',
                    'item_id', 'dept_id', 'cat_id','date','wm_yr_wk','d', \
                    'release', 'sell_price', 'price_max', 'price_min', \
                    'price_norm', 'price_nunique', 'item_nunique', \
                    'price_momentum', 'price_momentum_m', 'price_momentum_y', \
                    'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', \
                    'enc_cat_id_std', 'event_type_1', 'event_name_2', 'event_type_2', \
                    'enc_dept_id_std', 'enc_item_id_std', \
                    'rolling_std_7', 'rolling_std_14', \
                    'rolling_std_30', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', \
                    'rolling_mean_tmp_1_30', 'rolling_mean_tmp_7_7', \
                    'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30', \
                    'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', \
                    'rolling_mean_tmp_14_30']

# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    

    df = df[df['d']>=START_TRAIN]
    
    df = df[df['store_id']==store]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3

    state = "snap_" + store.split('_')[0]
    states = ['snap_CA','snap_TX','snap_WI']
    deleted_states = []
    for i in states:
        if i != state:
            deleted_states.append(i)
            
    features = [col for col in list(df) if (col not in remove_features and col not in deleted_states)]
    
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test

# split(trainning and validation)
def split(dataframe_for_splitting):
    train = dataframe_for_splitting[dataframe_for_splitting["d"] <= END_TRAIN - P_HORIZON]
    validation = dataframe_for_splitting[(dataframe_for_splitting["d"] <= END_TRAIN) & (dataframe_for_splitting["d"] > END_TRAIN - P_HORIZON)]
    return train, validation


import joblib
from sklearn.impute import SimpleImputer
import pandas as pd


for i in range(len(STORES_IDS)):
    # Load the saved model
    loaded_model = joblib.load(f'model_store_{STORES_IDS[i]}_GB.joblib')

    df, _ = get_data_by_store(STORES_IDS[i])
    df = pd.get_dummies(df, columns=['event_name_1'])
    columns_to_remove = ['id']
    features = list(df.columns)
    for col in columns_to_remove:
        features.remove(col)
    test = df[df["d"] > END_TRAIN]
    id = test["id"]
    d = test["d"]
    features.remove('d')
    X_test = test[features]
    X_test = X_test.drop(columns=['sales'])
    
    imputer = SimpleImputer(strategy='median')  # You can choose 'mean', 'median', 'most_frequent', or a constant value
    X_test = imputer.fit_transform(X_test)
    # Use the loaded model for prediction
    prediction = loaded_model.predict(X_test)

    print(f"--Predition of {STORES_IDS[i]}--")
    print(id)
    print(prediction)

    temp_df = pd.DataFrame({'id': id,'d': d, 'prediction': prediction})
    temp_df.to_csv(f"results_GB_{STORES_IDS[i]}.csv",index=False)
    print(f"Predicted Time Ahead: {len(temp_df)}")
    
    
import pandas as pd

df1 = pd.read_csv("results_GB_CA_1.csv")

for i in range(1,len(STORES_IDS)):    
    df2 = pd.read_csv(f"results_GB_{STORES_IDS[i]}.csv")

# Concatenate the two DataFrames vertically (along the rows)
    df1 = pd.concat([df1, df2], ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
df1.to_csv("concatenated_file.csv", index=False)

df1 = pd.read_csv("concatenated_file.csv")

pivot_table = df1.pivot_table(index='id', columns='d', values='prediction', aggfunc='median')

pivot_table.to_csv("pivot_file.csv", index=False)

In [ ]:
import pandas as pd
# Save the concatenated DataFrame to a new CSV file
new_column_names = []
for i in range(28):
    new_column_names.append(f"F{i+1}")

df1 = pd.read_csv("concatenated_file.csv")

pivot_table = df1.pivot_table(index='id', columns='d', values='prediction', aggfunc='median')
pivot_table.columns = new_column_names

pivot_table.to_csv("pivot_file.csv")

import pandas as pd

# Read the first CSV file
df1 = pd.read_csv("datasets\sample_submission.csv")

# Read the second CSV file
df2 = pd.read_csv("pivot_file.csv")

# Merge the two DataFrames on the 'id' column
merged_df = pd.merge(df1, df2, on='id', how="left", suffixes=('_original', ''))

# Replace values in columns F1 to F28 in df1 with corresponding values from df2
for col in df1.columns[1:]:
    merged_df[col+'_original'] = merged_df[col+'_original'].fillna(merged_df[col])

# Drop the extra columns
merged_df.drop(columns=[col+'_original' for col in df1.columns[1:]], inplace=True)
merged_df.fillna(0, inplace=True)
# Save the updated DataFrame to a new CSV file
merged_df.to_csv("merged_file.csv", index=False)
